# Data Cleaning
## Introduction and Problem Statement
The goal of this project is to analyze the similarities between top-scoring questions posted on Stack Overflow, which are [calculated](https://meta.stackexchange.com/questions/229255/what-is-the-score-of-a-post) by the number of upvotes minus downvotes. The benefits of this analysis include helping users on Stack Overflow use language that is likely to get an answer and be helpful to others on the site, as well as increase a user's [reputation](https://stackoverflow.com/help/whats-reputation), which, for questions, is calculated by the number of upvotes received.

In [1]:
# data analysis and manipulation
import pandas as pd
import numpy as np

# files
import pickle

# text manipulation
import re
import string
from bs4 import BeautifulSoup

# natural language processing
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# topic modeling
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

### Getting the Data
This CSV file was created using a SQL query on Google Cloud Platform's BigQuery.
``` mysql
SELECT DISTINCT
  posts.id, title, body,
  answer_count, favorite_count, score,
  tags, view_count, reputation
FROM
  `bigquery-public-data.stackoverflow.stackoverflow_posts`
  AS posts
JOIN
  `bigquery-public-data.stackoverflow.users`
  AS users
  ON posts.id = users.id
WHERE
  post_type_id = 1
ORDER BY
  RAND()
LIMIT
  500000 
```
Features related to the project were selected from the public [dataset](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow?project=named-tube-302921&folder=&organizationId=) hosted on the site. To narrow the dataset further, I looked specifically at questions and took a random sample of 50,000 rows.

In [2]:
# read in the file to a dataframe
df = pd.read_csv('data/stackoverflow.csv')
df.head()

,id,title,body,answer_count,favorite_count,score,tags,view_count,reputation
0,10797100,How to split text by tags using PHP?,<p>I like to split a huge text by <code>&lt;li...,2,NaN,4,php|html|text|tags|split,246,1
1,4870140,Change java -Xmx after the VM has already started,<blockquote> <p><strong>Possible Duplicates:</...,1,NaN,1,java|debian|heap-memory|openjdk|jinfo,1352,1
2,7668298,.htaccess (mod_rewrite) recursion in handling ...,<p>My aim is to simulate virtual hosts using m...,1,NaN,0,apache|.htaccess|mod-rewrite,223,1
3,10582104,Running .NET MSBuild 3.5 tests on VS2010,<p>We have a solution that targets .NET Framew...,2,NaN,0,.net|visual-studio-2010|unit-testing|mstest,678,1
4,7489467,Remove capture groups from a regex,<p>I have a regex 'simple' that I'd like to us...,4,1.0,2,ruby|regex,1728,1


In [3]:
# get an idea of the dataframe size
df.shape

(500000, 9)

In [4]:
# look at the data types
df.dtypes

id                  int64
title              object
body               object
answer_count        int64
favorite_count    float64
score               int64
tags               object
view_count          int64
reputation          int64
dtype: object

## Initial Data Prep and Cleaning
Let's first try to reduce our dataset by looking for, and removing, any null values.

In [5]:
# check for null values
df.isna().sum()

id                     0
title                  0
body                   0
answer_count           0
favorite_count    335820
score                  0
tags                   2
view_count             0
reputation             0
dtype: int64

In [6]:
# drop null values
df.dropna(inplace=True)
df.shape

(164180, 9)

There shouldn't be any duplicates because we ensured the rows we pulled in with a SQL query in Google Cloud Platform's BigQuery were distinct, but let's check to be sure.

In [7]:
df.duplicated().sum()

0

#### Look at Top Tags
Later on, we'll be trying to identify topics based on our topic modeling. Stack Overflow uses tags, which we have data for, to group posts based on relevant topics. Let's try to reduce the dataset further by filtering for the top 100 tags. The `tags` column is an object type, which is a string, so we should be able to iterate through each row for the Tags column and get counts for each unique tag.

In [8]:
# replace the pipe with a space
df['tags'] = df.tags.str.replace('|', ' ', regex=False)
df.head()

,id,title,body,answer_count,favorite_count,score,tags,view_count,reputation
4,7489467,Remove capture groups from a regex,<p>I have a regex 'simple' that I'd like to us...,4,1.0,2,ruby regex,1728,1
6,1753336,hash['key'] to hash.key in Ruby,<p>I have a a hash</p> <pre><code>foo = {'bar'...,5,13.0,33,ruby activerecord hash,14477,1
9,14309337,How to implement has_many :through relationshi...,<p>i've been searching through similar questio...,1,1.0,0,ruby-on-rails-3 has-many-through,3362,1
10,8022453,JPA Update Query -- Why won't my transaction w...,<p>I am having a performance problem with Java...,1,1.0,2,jpa eclipselink derby,6594,1
16,11132919,Jquery paging scroll in Asp.net MVC,<p>I tried to search 2 days about how to do th...,1,0.0,0,jquery asp.net-mvc,2507,1


In [9]:
# get the counts for each unique tag
counts = df.tags.str.split(expand=True).stack().value_counts()
counts

c#                          16119
java                        14557
javascript                  11624
android                     11366
php                         10081
                            ...  
pythonanywhere                  1
textreader                      1
tuckey-urlrewrite-filter        1
image-rendering                 1
aqtime                          1
Length: 18382, dtype: int64

In [10]:
# get the number of unique tags
len(counts)

18382

In [11]:
# get the top 100 tags
top100 = counts[:100]
top100

c#            16119
java          14557
javascript    11624
android       11366
php           10081
              ...  
generics        595
math            589
templates       587
rest            581
parsing         580
Length: 100, dtype: int64

In [12]:
# convert the 100 most used tags to a list
top100_list = top100.index.tolist()
top100_list

['c#',
 'java',
 'javascript',
 'android',
 'php',
 'jquery',
 'python',
 'c++',
 'iphone',
 '.net',
 'ios',
 'objective-c',
 'html',
 'asp.net',
 'mysql',
 'ruby-on-rails',
 'css',
 'sql',
 'c',
 'ruby',
 'asp.net-mvc',
 'wpf',
 'linux',
 'django',
 'sql-server',
 'xcode',
 'xml',
 'windows',
 'database',
 'ajax',
 'regex',
 'algorithm',
 'eclipse',
 'arrays',
 'multithreading',
 'git',
 'json',
 'ruby-on-rails-3',
 'performance',
 'string',
 'r',
 'facebook',
 'osx',
 'asp.net-mvc-3',
 'html5',
 'cocoa',
 'cocoa-touch',
 'image',
 'spring',
 'wcf',
 'winforms',
 'visual-studio-2010',
 'linq',
 'ipad',
 'visual-studio',
 'delphi',
 'node.js',
 'vb.net',
 'bash',
 'security',
 'hibernate',
 'entity-framework',
 'web-services',
 'perl',
 'unit-testing',
 'scala',
 'oracle',
 'oop',
 'swing',
 'apache',
 'forms',
 'sql-server-2008',
 'debugging',
 'file',
 'google-app-engine',
 'qt',
 'sqlite',
 'api',
 'silverlight',
 'user-interface',
 'validation',
 'tsql',
 'winapi',
 'design-pattern

We'll now use our list of the top 100 tags to filter out rows in the DataFrame that don't feature any of those tags. First, we'll add a column containing only those tags within each row that appear in the most frequently used tags. Aftewards, we'll remove rows with empty lists from that top_tags_only column since those will be rows that had no top tags used in the Tags column.

In [13]:
# create a column with tags from the tags column that appear in the top 100 most frequently-used tags
df['top_tags_only'] = df['tags'].str.split().apply(lambda x : [y for y in x if y in top100_list])
df.head()

,id,title,body,answer_count,favorite_count,score,tags,view_count,reputation,top_tags_only
4,7489467,Remove capture groups from a regex,<p>I have a regex 'simple' that I'd like to us...,4,1.0,2,ruby regex,1728,1,"[ruby, regex]"
6,1753336,hash['key'] to hash.key in Ruby,<p>I have a a hash</p> <pre><code>foo = {'bar'...,5,13.0,33,ruby activerecord hash,14477,1,[ruby]
9,14309337,How to implement has_many :through relationshi...,<p>i've been searching through similar questio...,1,1.0,0,ruby-on-rails-3 has-many-through,3362,1,[ruby-on-rails-3]
10,8022453,JPA Update Query -- Why won't my transaction w...,<p>I am having a performance problem with Java...,1,1.0,2,jpa eclipselink derby,6594,1,[]
16,11132919,Jquery paging scroll in Asp.net MVC,<p>I tried to search 2 days about how to do th...,1,0.0,0,jquery asp.net-mvc,2507,1,"[jquery, asp.net-mvc]"


In [14]:
# remove rows with 0 top tags
df = df[df['top_tags_only'].apply(lambda x: len(x)) > 0]

In [15]:
# drop the top_tags_only column
df.drop(columns=['top_tags_only'], inplace=True)
df.shape

(140291, 9)

#### Look at Top Scores
Since we want to determine the kinds of questions posed on Stack Overflow that are considered helpful, let's take a look at the `score` column to see what the top 10% of scores are and what the bottom 10% of scores are. We'll use data related to the top 10% of scores for further analysis, and, if time permits, we'll also analyze the language that is perhaps unique to unhelpful questions.

In [16]:
# look at information related to the score
df.score.describe()

count    140291.000000
mean          8.454648
std          55.772535
min         -49.000000
25%           1.000000
50%           3.000000
75%           7.000000
max       14772.000000
Name: score, dtype: float64

In [17]:
# create column signifying whether the score for a post is in the top 10 percent
df['top_10_percent'] = (df['score'] >= df['score'].quantile(0.90)).astype(int)
df.head()

,id,title,body,answer_count,favorite_count,score,tags,view_count,reputation,top_10_percent
4,7489467,Remove capture groups from a regex,<p>I have a regex 'simple' that I'd like to us...,4,1.0,2,ruby regex,1728,1,0
6,1753336,hash['key'] to hash.key in Ruby,<p>I have a a hash</p> <pre><code>foo = {'bar'...,5,13.0,33,ruby activerecord hash,14477,1,1
9,14309337,How to implement has_many :through relationshi...,<p>i've been searching through similar questio...,1,1.0,0,ruby-on-rails-3 has-many-through,3362,1,0
16,11132919,Jquery paging scroll in Asp.net MVC,<p>I tried to search 2 days about how to do th...,1,0.0,0,jquery asp.net-mvc,2507,1,0
22,4377685,Moose method modifiers on DBIx::Class::Schema ...,<p>For any given result class MySchema::Result...,1,1.0,8,perl moose catalyst dbix-class,779,11,0


In [18]:
# create a DataFrame keeping only the top scoring posts
df_top = df[df['top_10_percent'].apply(lambda x: x == 1)].copy(deep=True)

In [19]:
# drop boolean column
df_top.drop(columns=['top_10_percent'], inplace=True)

In [20]:
# sort values by score
df_top.sort_values(by=['score'], ascending=False, inplace=True)

In [21]:
# reset the index
df_top.reset_index(drop=True, inplace=True)
df_top.head()

,id,title,body,answer_count,favorite_count,score,tags,view_count,reputation
0,11227809,Why is processing a sorted array faster than a...,<p>Here is a piece of C++ code that seems very...,13,7317.0,14772,java c++ performance optimization branch-predi...,805490,1
1,477816,What is the correct JSON content type?,<p>I've been messing around with <a href= http...,29,1089.0,6768,json content-type,1403837,95
2,244777,Can I use comments inside a JSON file?,<p>Can I use comments inside a JSON file? If s...,39,378.0,3437,json comments,631045,25
3,208105,How do I remove a property from a JavaScript o...,<p>Say I create an object as follows:</p> <pre...,13,539.0,2891,javascript object-properties,865544,16
4,271526,Avoiding != null statements,<p>The idiom I use the most when programming i...,49,1083.0,2499,java nullpointerexception null,737912,369


In [22]:
# repeat the process for lowest scoring 10 percent of posts
df['bottom_10_percent'] = (df['score'] <= df['score'].quantile(0.10)).astype(int)
df_bottom = df[df['bottom_10_percent'].apply(lambda x: x == 1)].copy(deep=True)
df_bottom.drop(columns=['bottom_10_percent'], inplace=True)
df_bottom.sort_values(by=['score'], ascending=False, inplace=True)
df_bottom.reset_index(drop=True, inplace=True)
df_bottom.head()

,id,title,body,answer_count,favorite_count,score,tags,view_count,reputation,top_10_percent
0,14309337,How to implement has_many :through relationshi...,<p>i've been searching through similar questio...,1,1.0,0,ruby-on-rails-3 has-many-through,3362,1,0
1,5402620,Android location update problem,<p>I am working on one application in android ...,1,0.0,0,android,1466,8785,0
2,7161565,Efficient way to load all contacts and all pho...,<blockquote> <p><strong>Possible Duplicate:</s...,0,0.0,0,android contacts,232,1,0
3,7692992,Custom routing in code ingniter,<p>I want to use codeigniter for an ecommerce ...,1,2.0,0,php codeigniter,128,1,0
4,9973140,Uploading file to sharepoint server,<p>I need to upload files from my PC to a remo...,1,1.0,0,c# sharepoint uploading,4172,1,0


In [23]:
# # get an idea of the dataframe size
df_top.shape

(14091, 9)

## Further Data Cleaning and Text Preprocessing
Let's now try to clean up text in the `body` column. We'll look at a row to get an idea of the characters that can be removed.

In [24]:
# look at a sample of text
df_top.loc[4].body

"<p>The idiom I use the most when programming in Java is to test if <code>object != null</code> before I use it. This is to avoid a <a href= http://docs.oracle.com/javase/6/docs/api/index.html?java/lang/NullPointerException.html >NullPointerException</a>. I find the code very ugly and it becomes unreadable.</p> <p>Is there a good alternative to this? </p> <p>I want to address the necessity to test every object if you want to access a field or method of this object. For example:</p> <pre><code>if (someobject != null) { someobject.doCalc(); } </code></pre> <p>In this case I will avoid a <code>NullPointerException</code> and I don't know exactly if the object is <code>null</code> or not. These tests appear throughout my code as a consequence.</p>"

In [25]:
def clean_text(text):
    '''
    This function makes text lowercase,
    removes numbers,
    removes HTML,
    removes punctuation,
    removes words with an underscore,
    removes words with non-ASCII characters,
    and removes words starting with two or more of the same
    consecutive characters
    '''
    
    text = text.lower()
    text = re.sub('\w*\d\w*', ' ', text)
    text = BeautifulSoup(text, "lxml").get_text()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('_+', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub(r'\b(.)\1{1,}', ' ', text)
    return text

cleaned = lambda x: clean_text(x)

In [26]:
# apply cleaning
df_top['body'] = df_top.body.apply(cleaned)
df_top.head()

,id,title,body,answer_count,favorite_count,score,tags,view_count,reputation
0,11227809,Why is processing a sorted array faster than a...,here is a piece of c code that seems very pecu...,13,7317.0,14772,java c++ performance optimization branch-predi...,805490,1
1,477816,What is the correct JSON content type?,ive been messing around with json for some tim...,29,1089.0,6768,json content-type,1403837,95
2,244777,Can I use comments inside a JSON file?,can i use comments inside a json file if so how,39,378.0,3437,json comments,631045,25
3,208105,How do I remove a property from a JavaScript o...,say i create an object as follows var myobject...,13,539.0,2891,javascript object-properties,865544,16
4,271526,Avoiding != null statements,the idiom i use the most when programming in j...,49,1083.0,2499,java nullpointerexception null,737912,369


In [27]:
# look again at the sample text
df_top.loc[4].body

'the idiom i use the most when programming in java is to test if object null before i use it this is to avoid a nullpointerexception i find the code very ugly and it becomes unreadable is there a good alternative to this i want to address the necessity to test every object if you want to access a field or method of this object for example if someobject null someobjectdocalc in this case i will avoid a nullpointerexception and i dont know exactly if the object is null or not these tests appear throughout my code as a consequence'

## Organizing the Data
We'll take the cleaned data with us for EDA later, along with topic modeling. This means two separate datasets are needed:
1. **Corpus**: a collection of text
2. **Document-Term Matrix**: word counts in matrix format

### Corpus
The corpus is just the `body` column of our DataFrame, but we'll take the entire dataset with us in case the data in the other columns is helpful in gaining some unique insights during our EDA.

In [28]:
# save for later use
df_top.to_pickle("df.pkl")

### Document-Term Matrix
We'll use two tools for our topic modeling, both of which convert text data to vectors for analysis. The Count Vectorizer will be beneficial for analyzing the counts of each term (i.e. the term frequency) while the TF-IDF Vectorizer will weigh the term frequency against how often the term appears across all documents (questions) in the dataset, which is the inverse document frequency. This allows us some flexibility when choosing the best model for our data.
#### Count Vectorizer

In [29]:
# instantiate a CV object
count_vectorizer = CountVectorizer(stop_words = 'english', max_features=50000)

In [30]:
# convert to a matrix
cv_wm = count_vectorizer.fit_transform(df_top.body)

In [31]:
# retrieve terms found in the corpus
cv_tokens = count_vectorizer.get_feature_names()
print(cv_tokens)

['aaa', 'ab', 'aba', 'abab', 'ababab', 'ababba', 'abaccddccefe', 'abacounter', 'abaddressbookcreate', 'abaddressbookcreatewithoptions', 'abaddressbookrequestaccesswithcompletion', 'abadressbookcreatewithoptions', 'abandonaudiofocus', 'abandoned', 'abandoning', 'abandonware', 'abatha', 'abb', 'abba', 'abbbccc', 'abbcdbef', 'abbot', 'abbots', 'abbr', 'abbreviate', 'abbreviated', 'abbreviation', 'abbreviationattribute', 'abbreviationattributestring', 'abbreviations', 'abby', 'abc', 'abcabca', 'abcbd', 'abcchildelementwithxlink', 'abcd', 'abcdabcd', 'abcdafghijkl', 'abcdc', 'abcdccom', 'abcdedd', 'abcdef', 'abcdefg', 'abcdefgh', 'abcdefghij', 'abcdefghijd', 'abcdefghijklmnopqrstuvwxyz', 'abcdefgm', 'abcdehead', 'abcdmodel', 'abcdyyyy', 'abcexe', 'abcfoodef', 'abcframe', 'abcgghh', 'abchydmycompanycom', 'abcifgh', 'abcmakewidgetsself', 'abcn', 'abcom', 'abcrootelement', 'abcxyz', 'abczxfghij', 'abd', 'abe', 'abefore', 'abeforehover', 'abeginnersguidetodependencyinjection', 'abernathy', 'abe

In [32]:
# create document-term matrix / dataframe
df_cv = pd.DataFrame(cv_wm.toarray(), columns=cv_tokens)
df_cv.index = df_top.index
df_cv.head()

,aaa,ab,aba,abab,ababab,ababba,abaccddccefe,abacounter,abaddressbookcreate,abaddressbookcreatewithoptions,...,zune,zur,zvfzjni,zx,zxf,zxing,zxvf,zxwhat,zxy,zyx
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# save the document-term matrix and count vectorizer object for later use
df_cv.to_pickle("df_cv.pkl")
outfile = open("cv.pkl",'wb')
pickle.dump(count_vectorizer, outfile)
outfile.close()

#### TF-IDF Vectorizer

#### TF-IDF Vectorizer

In [34]:
# repeat the same process for tf-idf vectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_wm = tfidf_vectorizer.fit_transform(df_top.body)
tfidf_tokens = tfidf_vectorizer.get_feature_names()

df_tfidf = pd.DataFrame(tfidf_wm.toarray(), columns=tfidf_tokens)
df_tfidf.index = df_top.index

df_tfidf.to_pickle("df_tfidf.pkl")
outfile = open("tfidf.pkl",'wb')
pickle.dump(tfidf_vectorizer, outfile)
outfile.close()